In [68]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [69]:
#get info from csv in weatherpy
cities_pd = pd.read_csv("../weatherpy/output_data/city_weather.csv")
cities_pd.head(10)


,Unnamed: 0,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
0,0,port alfred,64.72,83,26.89,-33.59,56,ZA,19.89,1603144715
1,1,antofagasta,59.00,82,-70.40,-23.65,90,CL,12.75,1603144715
2,2,yurimaguas,93.02,41,-76.08,-5.90,95,PE,3.71,1603144715
3,3,champasak,70.95,99,106.00,14.75,85,LA,0.72,1603144716
4,4,avarua,80.60,78,-159.78,-21.21,6,CK,6.93,1603144716
5,5,bluff,46.99,84,168.33,-46.60,91,NZ,14.00,1603144717
6,6,tel aviv-yafo,75.20,78,34.78,32.08,28,IL,5.82,1603144494
7,7,fortuna,62.01,87,-124.16,40.60,40,US,13.87,1603144717
8,8,teya,25.70,94,92.63,60.38,98,RU,7.81,1603144718
9,9,ushuaia,39.20,80,-68.30,-54.80,75,AR,35.57,1603144482


In [70]:
gmaps.configure(api_key=g_key)

In [75]:
# Plot Heatmap
fig = gmaps.figure()

# Store latitude, longitude and Humidity in locations
locations = cities_pd[["Latitude", "Longitude"]]
Humidity = cities_pd["Humidity"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [85]:
#create new data with specific conditions
ideal_cities = cities_pd[(cities_pd["Temperature"]>75)&(cities_pd["Temperature"]<80)&(cities_pd["Windspeed"]<10)&(cities_pd["Cloudiness"]<10)].dropna()
ideal_cities

,Unnamed: 0,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
104,104,corsicana,78.80,57,-96.47,32.10,1,US,8.05,1603144755
202,202,mandali,75.16,19,45.55,33.75,0,IQ,9.91,1603144812
225,225,dillon,75.20,60,-79.37,34.42,1,US,3.36,1603144824
226,226,bandarbeyla,77.68,76,50.81,9.49,0,SO,2.48,1603144825
228,228,aswan,78.30,26,32.91,24.09,0,EG,2.39,1603144826
255,255,avera,77.00,53,-82.53,33.19,1,US,6.93,1603144836
256,256,chake chake,76.50,86,39.77,-5.25,3,TZ,7.38,1603144837
288,288,durango,76.93,22,-104.83,24.83,0,MX,5.30,1603144849
296,296,sardarshahr,76.64,18,74.48,28.43,0,IN,7.02,1603144853
408,408,kisanga,75.60,73,37.70,-7.44,3,TZ,6.31,1603144897


In [86]:
#create new df *copy to avoid the error message
hotel_df = ideal_cities[["City","Country","Latitude", "Longitude"]].copy()
#add hotel name column
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
104,corsicana,US,32.10,-96.47,
202,mandali,IQ,33.75,45.55,
225,dillon,US,34.42,-79.37,
226,bandarbeyla,SO,9.49,50.81,
228,aswan,EG,24.09,32.91,
255,avera,US,33.19,-82.53,
256,chake chake,TZ,-5.25,39.77,
288,durango,MX,24.83,-104.83,
296,sardarshahr,IN,28.43,74.48,
408,kisanga,TZ,-7.44,37.70,


In [87]:
# find the closest hotels with params using gkey
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {  
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}
# use iterrows to iterate through dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    latitude = row['Latitude']
    longitude = row['Longitude']

    # add keyword to params dict
    params['location'] = f'{latitude},{longitude}'

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {row['City']} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 104: corsicana.
Closest corsicana hotel is Holiday Inn Express & Suites Corsicana I-45.
------------
Retrieving Results for Index 202: mandali.
Closest mandali hotel is بيت خالد امين.
------------
Retrieving Results for Index 225: dillon.
Closest dillon hotel is Dillon Express Inn.
------------
Retrieving Results for Index 226: bandarbeyla.
Closest bandarbeyla hotel is REER XOOSHLE GROUP.
------------
Retrieving Results for Index 228: aswan.
Closest aswan hotel is Nuba Nile.
------------
Retrieving Results for Index 255: avera.
Missing field/result... skipping.
------------
Retrieving Results for Index 256: chake chake.
Closest chake chake hotel is Hotel Archipelago.
------------
Retrieving Results for Index 288: durango.
Missing field/result... skipping.
------------
Retrieving Results for Index 296: sardarshahr.
Closest sardarshahr hotel is Hotel Moti Mahal.
------------
Retrieving Results for Index 408: kisanga.
Missing field/result... skipping.
--------

In [88]:
#df with set columns and name of hotels
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
104,corsicana,US,32.10,-96.47,Holiday Inn Express & Suites Corsicana I-45
202,mandali,IQ,33.75,45.55,بيت خالد امين
225,dillon,US,34.42,-79.37,Dillon Express Inn
226,bandarbeyla,SO,9.49,50.81,REER XOOSHLE GROUP
228,aswan,EG,24.09,32.91,Nuba Nile
255,avera,US,33.19,-82.53,
256,chake chake,TZ,-5.25,39.77,Hotel Archipelago
288,durango,MX,24.83,-104.83,
296,sardarshahr,IN,28.43,74.48,Hotel Moti Mahal
408,kisanga,TZ,-7.44,37.70,


In [89]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [90]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add layer
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))